<a href="https://colab.research.google.com/github/evroth/location-optimization/blob/main/Uber_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import requests
import json


def get_ride_price(origin_latitude, origin_longitude, destination_latitude, destination_longitude):
    url = "https://www.uber.com/api/loadFEEstimates?localeCode=en"

    payload = json.dumps({
      "origin": {
        "latitude": origin_latitude,
        "longitude": origin_longitude
      },
      "destination": {
        "latitude": destination_latitude,
        "longitude": destination_longitude
      },
      "locale": "en"
    })
    headers = {
      'content-type': 'application/json',
      'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36',
      'x-csrf-token': 'x'
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    result = {x['vehicleViewDisplayName']: x['fareString'] for x in response.json()['data']['prices']}

    # I want to add something to just reference the UberX price but it isn't always in the same location every time
    UberX = result['UberX']
    UberX= float(UberX[1:])  # remove the first character ('$') and convert to a float

    return UberX

In [2]:
print(get_ride_price(51.5072178, -0.1275862, 51.4974948, -0.1356583))

{'UberX': '£10.21', 'Assist': '£10.21', 'Access': '£10.21', 'Green': '£10.22', 'Comfort': '£12.45', 'UberXL': '£14.16', 'Uber Pet': '£15.35', 'Exec': '£16.75', 'Lux': '£22.47'}


In [3]:
result = {'UberX': '£10.21', 'Assist': '£10.21', 'Access': '£10.21', 'Green': '£10.22', 'Comfort': '£12.45', 'UberXL': '£14.16', 'Uber Pet': '£15.35', 'Exec': '£16.75', 'Lux': '£22.47'}

In [7]:
# figuring out how to reference
a = result['UberX']
a = float(a[1:])
print(a)

10.21


In [9]:
print(get_ride_price(47.447277, -122.3039276, 47.61828319569383, -122.19681018164148))

57.14


Trying to make it so this function can work on our data frames

In [11]:
import pandas as pd

# create the data for the dataframe
data = {
    'number': [1, 2],
    'origin_latitude': [37.7749, 40.7128],
    'origin_longitude': [-122.4194, -74.0060],
    'destination_latitude': [37.80274, 40.735497],
    'destination_longitude': [-122.405819, -73.991876]
}

# create the dataframe
df = pd.DataFrame(data)

# print the dataframe
print(df)

# 40.73549701575964, -73.99187619577094

   number  origin_latitude  origin_longitude  destination_latitude  \
0       1          37.7749         -122.4194             37.802740   
1       2          40.7128          -74.0060             40.735497   

   destination_longitude  
0            -122.405819  
1             -73.991876  


In [12]:
# apply the get_ride_price function to each row of the dataframe
df['ride_price'] = df.apply(lambda row: get_ride_price(row['origin_latitude'], row['origin_longitude'], 
                                                      row['destination_latitude'], row['destination_longitude']), axis=1)
print(df)

   number  origin_latitude  origin_longitude  destination_latitude  \
0       1          37.7749         -122.4194             37.802740   
1       2          40.7128          -74.0060             40.735497   

   destination_longitude  ride_price  
0            -122.405819       14.95  
1             -73.991876       24.29  
